In [2]:
import struct

DR = RealField(53)

DD = RealField(190)

def double_to_hex(f):
    packed = struct.pack('>d', float(f))
    return '0x' + packed.hex()

def float_to_hex(f):
    packed = struct.pack('>f', float(f))  # pack as 32-bit float (big endian)
    return '0x' + packed.hex()

def split_double_double(x):
    # Split RR value x into hi + lo (double-double)
    x = RealField(190)(x).exact_rational()
    x_hi = DR(x)  # convert to f64
    x_lo = x - DD(x_hi)
    return (x_lo,x_hi)

def split_triple_double(x):
    # Split RR value x into hi + lo (double-double)
    x_hi = DR(x)  # convert to f64
    x_mid = DR(x - DD(x_hi))
    x_lo = x - DD(x_hi) - DD(x_mid)
    return (x_lo, x_mid, x_hi)

def print_double_double(mark, x):
    splat = split_double_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}),")

def print_triple_double(mark, x):
    splat = split_triple_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}, {double_to_hex(splat[2])}),")

In [3]:
from sage.all import *
from mpmath import mp, chebyfit, besseli

mp.prec = 350

def horner_eval(x, coeffs):
    result = 0
    for c in coeffs:
        result = RealField(53)(result) * RealField(53)(x) + RealField(53)(c)
    return result

items = 16

terms = 39

print(f"pub(crate) static I0F_POLYS: [[u64; {terms}]; {items}] = [")

for i in range(items):
    f = lambda x: besseli(0, x)

    cf = chebyfit(f, [7.5 + i*0.5, 8.0 + i*0.5], terms)

    print("[")

    for c in reversed(cf):
        # print(c)
        print(double_to_hex(RealField(300)(c)) +",")

    print("],")
    
print("];")

# print(cf)

# Evaluate and test
# x = mp.mpf('8.25')
# print(f"Exact     I0({x}) = {f(x)}")
# print(f"Chebyfun  I0({x}) ≈ {horner_eval(x, cf)}")
# err = abs(f(x) - horner_eval(x, cf))
# print(f"Abs error         = {-RealField(70)(err).log2()}")


pub(crate) static I0F_POLYS: [[u64; 39]; 16] = [
[
0x3ff0000000122e26,
0xbe16c9f42ab7e95f,
0x3fd00000037a7917,
0xbe360d45339e182d,
0x3f90000065fdc17a,
0xbe36ecaddb8bc445,
0x3f3c71d7ca969145,
0xbe2430101eb89d74,
0x3edc7312b17256cd,
0xbe02501379d76e32,
0x3e72426d26bff243,
0xbdd30aae13f7fc48,
0x3e00e4a8f9885034,
0xbd9837aff2d93c73,
0x3d9052aac4ad91f6,
0xbd53a06d522b581a,
0x3d31b0d6080cec44,
0xbd04cc72d931fe76,
0x3cda7780286b9606,
0xbcad3d046193778a,
0x3c7ddc8eef7cca70,
0xbc4b676fa26f7b3b,
0x3c16c8881e75930a,
0xbbe110dbad899076,
0x3ba70711d09628c5,
0xbb6be4bfdf437d2e,
0x3b2e3e07c4e25b32,
0xbaed3836820798a7,
0x3aa90736a814ac9c,
0xba62e0d274070e0a,
0x3a18de861ec7b752,
0xb9cc4d45c2c424a9,
0x397b6fed27817a38,
0xb9263cb2b33ad34a,
0x38cd5be67c600e66,
0xb86e61e5720468fa,
0x38072dae09144e7a,
0xb7974616b7657578,
0x3717443dc29a1cf8,
],
[
0x3ff0000001526426,
0xbe58e59af6a26e3c,
0x3fd000003919d1ee,
0xbe753f4aaa2574a1,
0x3f900005c4900ba5,
0xbe737abc05a92a42,
0x3f3c729c1278ddaf,
0xbe5e412f3cd99b00,
0x3e

In [9]:
from mpmath import mp, besseli, linspace, fabs

print(besseli(0, 7.4015625000126715))

f = lambda x: besseli(0, x)
cf = chebyfit(f, [14.5, 15.0], 37)

xs = linspace(14.5, 15.0, 10000)
errors = [fabs(f(x) - horner_eval(x, cf)) for x in xs]

# Get max error and location
errors = []
for x in xs:
    actual = f(x)
    approx = horner_eval(x, cf)
    rel_err = fabs(approx - actual) / fabs(actual)
    errors.append(rel_err)

# Max error and its location
max_err = max(errors)
x_at_max = xs[errors.index(max_err)]

print(f"Max relative error = {-RealField(70)(max_err).log2()}")
print(f"At x = {x_at_max}")


244.6962646579708484500544101774564739233990936461021517401582798487201487523268303232062991732192169493
Max relative error = 19.384565016680689984
At x = 14.9967996799679967996799679967996799679967996799679967996799679967996799679967996799679967996799679968
